In [1]:
import pandas as pd
import numpy as np
import os
import heapq
import scipy.sparse as sparse

In [2]:
class Pre_Processer:
    def read_data_small(self):
        self.matrix_size = "500"
        self.data_users = pd.read_csv("../StatistaUsers500.csv")
        self.data_tracking = pd.read_csv("../StatistaTracking500.csv")
        
    def read_data_full(self):
        self.matrix_size = "10000"
        self.data_users = pd.read_csv("../StatistaUsers10000.csv")
        self.data_tracking = pd.read_csv("../StatistaTracking10000.csv")
        
    def get_size(self, exponent):
        if exponent == 0:
            return "extra_small"
        elif exponent == 0.5:
            return "small"
        elif exponent == 1:
            return "medium"
        elif exponent == 2:
            return "large"
        elif exponent == 999:
            return "full"
        else:
            return "different_size"           
        
    def create_matrix(self, exponent):
        # v2.0 Get all the viewed content for each user
        size = 100*10**exponent
        
        frame = self.data_tracking[["idUser", "idContent"]]
        frame["views"] = np.ones([len(frame["idContent"]),1])
        frame = frame.groupby(by=["idUser", "idContent"]).sum().reset_index()
        
        user = list(np.sort(frame.idUser.unique()))
        content = list(np.sort(frame.idContent.unique()))
        views = list(frame.views)
        rows = frame.idUser.astype('category', categories = user).cat.codes 
        cols = frame.idContent.astype('category', categories = content).cat.codes 
        
        self.sparsity_ofdata = 1 - (len(views) / (len(content) * len(user)))
                
        self.matrix = sparse.csr_matrix((views, (rows, cols)), shape=(len(user), len(content)))
        size_name = self.get_size(exponent)
        sparse.save_npz(f"../ratings_matrix_{self.matrix_size}_{size_name}", self.matrix, compressed=True)
        
        self.matrix_asFrame = frame
        self.matrix_asFrame.to_excel(f"../data_{self.matrix_size}_{size_name}.xlsx")

In [3]:
Data = Pre_Processer()

In [4]:
# Full pre-processing step 1. Optional: set_ratings_size(). If not set 150*10**3 = 150.000 rows of tracking data will be read.
Data.read_data_small()
Data.create_matrix(999)

C:\Users\studsmi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\studsmi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [99]:
Data.matrix_asFrame.describe()

,idUser,idContent,views
count,424911.000000,4.249110e+05,424911.000000
mean,96176.332578,3.104636e+05,3.809744
std,38568.482843,1.813573e+06,49.548521
min,5731.000000,0.000000e+00,1.000000
25%,64433.000000,5.690200e+04,1.000000
50%,121126.000000,1.947490e+05,2.000000
75%,121480.000000,3.256560e+05,3.000000
max,331436.000000,9.006080e+07,15666.000000


In [100]:
Data.matrix_asFrame.head()

,idUser,idContent,views
0,5731,0,12759.0
1,5731,1,1.0
2,5731,5,1.0
3,5731,17,24.0
4,5731,18,1.0


In [101]:
Data.matrix_asFrame[Data.matrix_asFrame.idUser == 5731].head()

,idUser,idContent,views
0,5731,0,12759.0
1,5731,1,1.0
2,5731,5,1.0
3,5731,17,24.0
4,5731,18,1.0


In [106]:
Data.matrix.toarray()

array([[1.2759e+04, 1.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [2.2000e+01, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [6.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00]])